In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/_main_/GoForward_/i3D_model/1c3d_2016

/content/drive/My Drive/_main_/GoForward_/i3D_model/1c3d_2016


In [0]:
# !pip install keras-gpu==2.2.0

In [0]:
# !pip install tensorflow-gpu==1.8.0

In [3]:
%tensorflow_version 1.x
import tensorflow as tf
tf.__version__

TensorFlow 1.x selected.


'1.15.2'

In [4]:
import keras

Using TensorFlow backend.


In [5]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
import os
import glob
import time
import sys
import warnings

import numpy as np
import pandas as pd

import keras
from keras import backend as K

from datagenerator import VideoClasses, FramesGenerator
from model_i3d import Inception_Inflated3d, add_i3d_top

In [0]:
from keras.models import Model, load_model
from keras import layers
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import Conv3D
from keras.layers import MaxPooling3D
from keras.layers import AveragePooling3D
from keras.layers import Dropout
from keras.layers import Reshape
from keras.layers import Lambda
from keras.layers import GlobalAveragePooling3D

In [0]:
from keras.engine.topology import get_source_inputs
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K

In [0]:
def layers_freeze(keModel:keras.models.Model) -> keras.models.Model:
    
    print("Freeze all %d layers in Model %s" % (len(keModel.layers), keModel.name))
    for layer in keModel.layers:
        layer.trainable = False

    return keModel

def layers_unfreeze(keModel:keras.models.Model) -> keras.models.Model:
    
    print("Unfreeze all %d layers in Model %s" % (len(keModel.layers), keModel.name))
    for layer in keModel.layers:
        layer.trainable = True

    return keModel

In [0]:
def count_params(keModel:keras.models.Model):

    trainable_count = int(
        np.sum([K.count_params(p) for p in set(keModel.trainable_weights)]))
    non_trainable_count = int(
        np.sum([K.count_params(p) for p in set(keModel.non_trainable_weights)]))

    print('Total params: {:,}'.format(trainable_count + non_trainable_count))
    print('Trainable params: {:,}'.format(trainable_count))
    print('Non-trainable params: {:,}'.format(non_trainable_count))
    
    return

In [0]:
def train_I3D_oflow_end2end(diVideoSet):
    """ 
    * Loads pretrained I3D model, 
    * reads optical flow data generated from training videos,
    * adjusts top-layers adequately for video data,
    * trains only news top-layers,
    * then fine-tunes entire neural network,
    * saves logs and models to disc.
    """
    
    """
    diVideoSet = {"sName" : "chalearn",
    "nClasses" : 249,   # number of classes
    "nFramesNorm" : 40,    # number of frames per video
    "nMinDim" : 240,   # smaller dimension of saved video-frames
    "tuShape" : (240, 320), # height, width
    "nFpsAvg" : 10,
    "nFramesAvg" : 50, 
    "fDurationAvG" : 5.0} # seconds 
    """
   
    # directories
    sFolder = "%03d-%d"%(diVideoSet["nClasses"], diVideoSet["nFramesNorm"])
    sClassFile       = "data-set/{}/{}/class.csv".format(diVideoSet["sName"], diVideoSet["nClasses"]) #, diVideoSet["nClasses"])
    #sClassFile       = "data-set/%s/%03d/class.csv"%(diVideoSet["sName"], diVideoSet["nClasses"])
    #sVideoDir        = "data-set/%s/%03d"%(diVideoSet["sName"], diVideoSet["nClasses"])
    #sImageDir        = "data-temp/%s/%s/image"%(diVideoSet["sName"], sFolder)
    #sImageFeatureDir = "data-temp/%s/%s/image-i3d"%(diVideoSet["sName"], sFolder)
    #sOflowDir        = "data-temp/%s/%s/oflow"%(diVideoSet["sName"], sFolder)
    sOflowDir        = "data-set/{}".format(diVideoSet["sName"], sFolder)
    #print(sOflowDir)
    #sOflowFeatureDir = "data-temp/%s/%s/oflow-i3d"%(diVideoSet["sName"], sFolder)
    
    sModelDir        = "model"

    diTrainTop = {
        "fLearn" : 1e-3,
        "nEpochs" : 3}

    diTrainAll = {
        "fLearn" : 1e-4,
        "nEpochs" : 17}

    nBatchSize = 4

    print("\nStarting I3D end2end training ...")
    print(os.getcwd())

    # read the ChaLearn classes
    oClasses = VideoClasses(sClassFile)
    #print(oClasses.liClasses)

    # Load training data
    genFramesTrain = FramesGenerator(sPath=sOflowDir + "/train", 
                                     nBatchSize=nBatchSize, 
                                     nFrames=diVideoSet["nFramesNorm"], 
                                     nHeight=224, 
                                     nWidth=224, 
                                     nChannels=3, 
                                     liClassesFull=oClasses.liClasses)
    
    genFramesVal = FramesGenerator(sPath=sOflowDir + "/valid", 
                                   nBatchSize=nBatchSize, 
                                   nFrames=diVideoSet["nFramesNorm"], 
                                   nHeight=224, 
                                   nWidth=224, 
                                   nChannels=3, 
                                   liClassesFull=oClasses.liClasses)

    # Load pretrained i3d model and adjust top layer 
    print("Load pretrained I3D flow model ...")
    keI3DOflow = Inception_Inflated3d(include_top=False,
                                      weights='flow_imagenet_and_kinetics',
                                      input_shape=(diVideoSet["nFramesNorm"], 
                                                   224, 
                                                   224, 
                                                   3)
                                      )
    
    print("Add top layers with %d output classes ..." % oClasses.nClasses)
    keI3DOflow = layers_freeze(keI3DOflow)
    keI3DOflow = add_i3d_top(keI3DOflow, oClasses.nClasses, dropout_prob=0.5)
        
    # Prep logging
    sLog = time.strftime("%Y%m%d-%H%M", time.gmtime()) + \
        "-%s%03d-oflow-i3d"%(diVideoSet["sName"], diVideoSet["nClasses"])
    
    # Helper: Save results
    csv_logger = keras.callbacks.CSVLogger("log/" + sLog + "-acc.csv", append = True)

    # Helper: Save the model
    os.makedirs(sModelDir, exist_ok=True)
    cpTopLast = keras.callbacks.ModelCheckpoint(filepath = sModelDir + "/" + sLog + "-above-last.h5", 
                                                verbose = 0)
    
    cpTopBest = keras.callbacks.ModelCheckpoint(filepath = sModelDir + "/" + sLog + "-above-best.h5",
                                                verbose = 1, save_best_only = True)
    
    cpAllLast = keras.callbacks.ModelCheckpoint(filepath = sModelDir + "/" + sLog + "-entire-last.h5", 
                                                verbose = 0)
    
    cpAllBest = keras.callbacks.ModelCheckpoint(filepath = sModelDir + "/" + sLog + "-entire-best.h5",
                                                verbose = 1, save_best_only = True)

    # Fit top layers
    print("Fit I3D top layers with generator: %s" % (diTrainTop))
    optimizer = keras.optimizers.Adam(lr = diTrainTop["fLearn"])
    keI3DOflow.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    count_params(keI3DOflow)    

    keI3DOflow.fit_generator(generator = genFramesTrain,
                             validation_data = genFramesVal,
                             epochs = diTrainTop["nEpochs"],
                             workers = 4,
                             use_multiprocessing = True,
                             max_queue_size = 8,
                             verbose = 1,
                             callbacks=[csv_logger, 
                                        cpTopLast, 
                                        cpTopBest])
    
    # Fit entire I3D model
    print("Finetune all I3D layers with generator: %s" % (diTrainAll))
    keI3DOflow = layers_unfreeze(keI3DOflow)
    optimizer = keras.optimizers.Adam(lr = diTrainAll["fLearn"])
    keI3DOflow.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    count_params(keI3DOflow)    

    keI3DOflow.fit_generator(generator = genFramesTrain, 
                             validation_data = genFramesVal,
                             epochs = diTrainAll["nEpochs"],
                             workers = 4,                 
                             use_multiprocessing = True,
                             max_queue_size = 8, 
                             verbose = 1,
                             callbacks=[csv_logger, 
                                        cpAllLast, 
                                        cpAllBest])

In [0]:
diVideoSet = {'sName': 'chalearn_img_flow_tvl1-fast',
 'nClasses': 3,
 'nFramesNorm': 120,
 'nMinDim': 224,
 'tuShape': (224, 224),
 'nFpsAvg': 30,
 'nFramesAvg': 120,
 'fDurationAvg': 4.0}

In [19]:
# directories
sFolder = "%03d-%d"%(diVideoSet["nClasses"], diVideoSet["nFramesNorm"])
print('sFolder:', sFolder)
sClassFile       = "data-set/{}/{}/class.csv".format(diVideoSet["sName"], diVideoSet["nClasses"]) #, diVideoSet["nClasses"])
print("sClassFile:", sClassFile)
#sClassFile       = "data-set/%s/%03d/class.csv"%(diVideoSet["sName"], diVideoSet["nClasses"])
#sVideoDir        = "data-set/%s/%03d"%(diVideoSet["sName"], diVideoSet["nClasses"])
#sImageDir        = "data-temp/%s/%s/image"%(diVideoSet["sName"], sFolder)
#sImageFeatureDir = "data-temp/%s/%s/image-i3d"%(diVideoSet["sName"], sFolder)
#sOflowDir        = "data-temp/%s/%s/oflow"%(diVideoSet["sName"], sFolder)
sOflowDir        = "data-set/{}".format(diVideoSet["sName"], sFolder)
print("sOflowDir:", sOflowDir)
#print(sOflowDir)
#sOflowFeatureDir = "data-temp/%s/%s/oflow-i3d"%(diVideoSet["sName"], sFolder)

sModelDir        = "model"

sFolder: 003-120
sClassFile: data-set/chalearn_img_flow_tvl1-fast/3/class.csv
sOflowDir: data-set/chalearn_img_flow_tvl1-fast


In [20]:
import os
sModelDir        = "model"

diTrainTop = {
    "fLearn" : 1e-3,
    "nEpochs" : 3}

diTrainAll = {
    "fLearn" : 1e-4,
    "nEpochs" : 17}

nBatchSize = 4

print("\nStarting I3D end2end training ...")
print(os.getcwd())


Starting I3D end2end training ...
/content/drive/My Drive/_main_/GoForward_/i3D_model/1c3d_2016


In [26]:
# read the ChaLearn classes
oClasses = VideoClasses(sClassFile)
print(oClasses.liClasses)

Loaded 3 classes from data-set/chalearn_img_flow_tvl1-fast/3/class.csv
['c001', 'c002', 'c003']


In [27]:
os.listdir("data-set/chalearn_img_flow_tvl1-fast/train")

['c001', 'c002', 'c003']

In [28]:
# Load training data
genFramesTrain = FramesGenerator(sPath=sOflowDir + "/train", 
                                  nBatchSize=nBatchSize, 
                                  nFrames=diVideoSet["nFramesNorm"], 
                                  nHeight=224, 
                                  nWidth=224, 
                                  nChannels=3, 
                                  liClassesFull=oClasses.liClasses)

self.dfVideos:                                             sFrameDir
0   data-set/chalearn_img_flow_tvl1-fast/train/c00...
1   data-set/chalearn_img_flow_tvl1-fast/train/c00...
2   data-set/chalearn_img_flow_tvl1-fast/train/c00...
3   data-set/chalearn_img_flow_tvl1-fast/train/c00...
4   data-set/chalearn_img_flow_tvl1-fast/train/c00...
5   data-set/chalearn_img_flow_tvl1-fast/train/c00...
6   data-set/chalearn_img_flow_tvl1-fast/train/c00...
7   data-set/chalearn_img_flow_tvl1-fast/train/c00...
8   data-set/chalearn_img_flow_tvl1-fast/train/c00...
9   data-set/chalearn_img_flow_tvl1-fast/train/c00...
10  data-set/chalearn_img_flow_tvl1-fast/train/c00...
11  data-set/chalearn_img_flow_tvl1-fast/train/c00...
12  data-set/chalearn_img_flow_tvl1-fast/train/c00...
13  data-set/chalearn_img_flow_tvl1-fast/train/c00...
14  data-set/chalearn_img_flow_tvl1-fast/train/c00...
15  data-set/chalearn_img_flow_tvl1-fast/train/c00...
16  data-set/chalearn_img_flow_tvl1-fast/train/c00...
17  data-set/

In [29]:
genFramesVal = FramesGenerator(sPath=sOflowDir + "/valid", 
                                nBatchSize=nBatchSize, 
                                nFrames=diVideoSet["nFramesNorm"], 
                                nHeight=224, 
                                nWidth=224, 
                                nChannels=3, 
                                liClassesFull=oClasses.liClasses)

self.dfVideos:                                             sFrameDir
0   data-set/chalearn_img_flow_tvl1-fast/valid/c00...
1   data-set/chalearn_img_flow_tvl1-fast/valid/c00...
2   data-set/chalearn_img_flow_tvl1-fast/valid/c00...
3   data-set/chalearn_img_flow_tvl1-fast/valid/c00...
4   data-set/chalearn_img_flow_tvl1-fast/valid/c00...
5   data-set/chalearn_img_flow_tvl1-fast/valid/c00...
6   data-set/chalearn_img_flow_tvl1-fast/valid/c00...
7   data-set/chalearn_img_flow_tvl1-fast/valid/c00...
8   data-set/chalearn_img_flow_tvl1-fast/valid/c00...
9   data-set/chalearn_img_flow_tvl1-fast/valid/c00...
10  data-set/chalearn_img_flow_tvl1-fast/valid/c00...
11  data-set/chalearn_img_flow_tvl1-fast/valid/c00...
12  data-set/chalearn_img_flow_tvl1-fast/valid/c00...
13  data-set/chalearn_img_flow_tvl1-fast/valid/c00...
self.nSamples: 14
Detected 14 samples in data-set/chalearn_img_flow_tvl1-fast/valid ...
0     c001
1     c001
2     c001
3     c001
4     c001
5     c002
6     c002
7     c002

In [30]:
frames_ = diVideoSet["nFramesNorm"]
frames_

120

In [0]:
# path to pretrained models with top (classification layer)
WEIGHTS_PATH = {
    'rgb_kinetics_only' : 'https://github.com/dlpbc/keras-kinetics-i3d/releases/download/v0.2/rgb_inception_i3d_kinetics_only_tf_dim_ordering_tf_kernels.h5',
    'flow_kinetics_only' : 'https://github.com/dlpbc/keras-kinetics-i3d/releases/download/v0.2/flow_inception_i3d_kinetics_only_tf_dim_ordering_tf_kernels.h5',
    'rgb_imagenet_and_kinetics' : 'https://github.com/dlpbc/keras-kinetics-i3d/releases/download/v0.2/rgb_inception_i3d_imagenet_and_kinetics_tf_dim_ordering_tf_kernels.h5',
    'flow_imagenet_and_kinetics' : 'https://github.com/dlpbc/keras-kinetics-i3d/releases/download/v0.2/flow_inception_i3d_imagenet_and_kinetics_tf_dim_ordering_tf_kernels.h5'
}

# path to pretrained models with no top (no classification layer)
WEIGHTS_PATH_NO_TOP = {
    'rgb_kinetics_only' : 'https://github.com/dlpbc/keras-kinetics-i3d/releases/download/v0.2/rgb_inception_i3d_kinetics_only_tf_dim_ordering_tf_kernels_no_top.h5',
    'flow_kinetics_only' : 'https://github.com/dlpbc/keras-kinetics-i3d/releases/download/v0.2/flow_inception_i3d_kinetics_only_tf_dim_ordering_tf_kernels_no_top.h5',
    'rgb_imagenet_and_kinetics' : 'https://github.com/dlpbc/keras-kinetics-i3d/releases/download/v0.2/rgb_inception_i3d_imagenet_and_kinetics_tf_dim_ordering_tf_kernels_no_top.h5',
    'flow_imagenet_and_kinetics' : 'https://github.com/dlpbc/keras-kinetics-i3d/releases/download/v0.2/flow_inception_i3d_imagenet_and_kinetics_tf_dim_ordering_tf_kernels_no_top.h5'
}

In [0]:
include_top=False
weights='flow_imagenet_and_kinetics'
input_shape=(frames_, 224, 224, 3)
input_tensor = None

In [0]:
WEIGHTS_NAME = ['rgb_kinetics_only', 'flow_kinetics_only', 'rgb_imagenet_and_kinetics', 'flow_imagenet_and_kinetics']

In [0]:
if not (weights in WEIGHTS_NAME or weights is None or os.path.exists(weights)):
    raise ValueError('The `weights` argument should be either '
                      '`None` (random initialization) or %s' % 
                      str(WEIGHTS_NAME) + ' ' 
                      'or a valid path to a file containing `weights` values')

if weights in WEIGHTS_NAME and include_top and classes != 400:
    raise ValueError('If using `weights` as one of these %s, with `include_top`'
                      ' as true, `classes` should be 400' % str(WEIGHTS_NAME))

In [0]:
def conv3d_bn(x,
              filters,
              num_frames,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1, 1),
              use_bias = False,
              use_activation_fn = True,
              use_bn = True,
              name=None):
    """Utility function to apply conv3d + BN.

    # Arguments
        x: input tensor.
        filters: filters in `Conv3D`.
        num_frames: frames (time depth) of the convolution kernel.
        num_row: height of the convolution kernel.
        num_col: width of the convolution kernel.
        padding: padding mode in `Conv3D`.
        strides: strides in `Conv3D`.
        use_bias: use bias or not  
        use_activation_fn: use an activation function or not.
        use_bn: use batch normalization or not.
        name: name of the ops; will become `name + '_conv'`
            for the convolution and `name + '_bn'` for the
            batch norm layer.

    # Returns
        Output tensor after applying `Conv3D` and `BatchNormalization`.
    """
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None

    x = Conv3D(
        filters, (num_frames, num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=use_bias,
        name=conv_name)(x)

    if use_bn:
        if K.image_data_format() == 'channels_first':
            bn_axis = 1
        else:
            bn_axis = 4
        x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)

    if use_activation_fn:
        x = Activation('relu', name=name)(x)

    return x

In [0]:
def _obtain_input_shape(input_shape,
                        default_frame_size,
                        min_frame_size,
                        default_num_frames,
                        min_num_frames,
                        data_format,
                        require_flatten,
                        weights=None):
    """Internal utility to compute/validate the model's input shape.
    (Adapted from `keras/applications/imagenet_utils.py`)

    # Arguments
        input_shape: either None (will return the default network input shape),
            or a user-provided shape to be validated.
        default_frame_size: default input frames(images) width/height for the model.
        min_frame_size: minimum input frames(images) width/height accepted by the model.
        default_num_frames: default input number of frames(images) for the model.
        min_num_frames: minimum input number of frames accepted by the model.
        data_format: image data format to use.
        require_flatten: whether the model is expected to
            be linked to a classifier via a Flatten layer.
        weights: one of `None` (random initialization)
            or 'kinetics_only' (pre-training on Kinetics dataset).
            or 'imagenet_and_kinetics' (pre-training on ImageNet and Kinetics datasets).
            If weights='kinetics_only' or weights=='imagenet_and_kinetics' then
            input channels must be equal to 3.

    # Returns
        An integer shape tuple (may include None entries).

    # Raises
        ValueError: in case of invalid argument values.
    """
    if weights != 'kinetics_only' and weights != 'imagenet_and_kinetics' and input_shape and len(input_shape) == 4:
        if data_format == 'channels_first':
            if input_shape[0] not in {1, 3}:
                warnings.warn(
                    'This model usually expects 1 or 3 input channels. '
                    'However, it was passed an input_shape with ' +
                    str(input_shape[0]) + ' input channels.')
            default_shape = (input_shape[0], default_num_frames, default_frame_size, default_frame_size)
        else:
            if input_shape[-1] not in {1, 3}:
                warnings.warn(
                    'This model usually expects 1 or 3 input channels. '
                    'However, it was passed an input_shape with ' +
                    str(input_shape[-1]) + ' input channels.')
            default_shape = (default_num_frames, default_frame_size, default_frame_size, input_shape[-1])
    else:
        if data_format == 'channels_first':
            default_shape = (3, default_num_frames, default_frame_size, default_frame_size)
        else:
            default_shape = (default_num_frames, default_frame_size, default_frame_size, 3)
    if (weights == 'kinetics_only' or weights == 'imagenet_and_kinetics') and require_flatten:
        if input_shape is not None:
            if input_shape != default_shape:
                raise ValueError('When setting`include_top=True` '
                                 'and loading `imagenet` weights, '
                                 '`input_shape` should be ' +
                                 str(default_shape) + '.')
        return default_shape

    if input_shape:
        if data_format == 'channels_first':
            if input_shape is not None:
                if len(input_shape) != 4:
                    raise ValueError(
                        '`input_shape` must be a tuple of four integers.')
                if input_shape[0] != 3 and (weights == 'kinetics_only' or weights == 'imagenet_and_kinetics'):
                    raise ValueError('The input must have 3 channels; got '
                                     '`input_shape=' + str(input_shape) + '`')

                if input_shape[1] is not None and input_shape[1] < min_num_frames:
                    raise ValueError('Input number of frames must be at least ' +
                                     str(min_num_frames) + '; got '
                                     '`input_shape=' + str(input_shape) + '`')

                if ((input_shape[2] is not None and input_shape[2] < min_frame_size) or
                   (input_shape[3] is not None and input_shape[3] < min_frame_size)):
                    raise ValueError('Input size must be at least ' +
                                     str(min_frame_size) + 'x' + str(min_frame_size) + '; got '
                                     '`input_shape=' + str(input_shape) + '`')
        else:
            if input_shape is not None:
                if len(input_shape) != 4:
                    raise ValueError(
                        '`input_shape` must be a tuple of four integers.')
                if input_shape[-1] != 3 and (weights == 'kinetics_only' or weights == 'imagenet_and_kinetics'):
                    raise ValueError('The input must have 3 channels; got '
                                     '`input_shape=' + str(input_shape) + '`')

                if input_shape[0] is not None and input_shape[0] < min_num_frames:
                    raise ValueError('Input number of frames must be at least ' +
                                     str(min_num_frames) + '; got '
                                     '`input_shape=' + str(input_shape) + '`')

                if ((input_shape[1] is not None and input_shape[1] < min_frame_size) or
                   (input_shape[2] is not None and input_shape[2] < min_frame_size)):
                    raise ValueError('Input size must be at least ' +
                                     str(min_frame_size) + 'x' + str(min_frame_size) + '; got '
                                     '`input_shape=' + str(input_shape) + '`')
    else:
        if require_flatten:
            input_shape = default_shape
        else:
            if data_format == 'channels_first':
                input_shape = (3, None, None, None)
            else:
                input_shape = (None, None, None, 3)
    if require_flatten:
        if None in input_shape:
            raise ValueError('If `include_top` is True, '
                             'you should specify a static `input_shape`. '
                             'Got `input_shape=' + str(input_shape) + '`')
    return input_shape

In [37]:
input_shape = _obtain_input_shape(
    input_shape,
    default_frame_size=224, 
    min_frame_size=32, 
    default_num_frames=64,
    min_num_frames=8,
    data_format=K.image_data_format(),
    require_flatten=include_top,
    weights=weights)
input_shape

(120, 224, 224, 3)

In [38]:
if input_tensor is None:
    img_input = Input(shape=input_shape)
else:
    if not K.is_keras_tensor(input_tensor):
        img_input = Input(tensor=input_tensor, shape=input_shape)
    else:
        img_input = input_tensor

img_input

<tf.Tensor 'input_1:0' shape=(?, 120, 224, 224, 3) dtype=float32>

In [0]:
if K.image_data_format() == 'channels_first':
    channel_axis = 1
else:
    channel_axis = 4

In [40]:
# Downsampling via convolution (spatial and temporal)
x = conv3d_bn(img_input, 64, 7, 7, 7, strides=(2, 2, 2), padding='same', name='Conv3d_1a_7x7')

# Downsampling (spatial only)
x = MaxPooling3D((1, 3, 3), strides=(1, 2, 2), padding='same', name='MaxPool2d_2a_3x3')(x)
x = conv3d_bn(x, 64, 1, 1, 1, strides=(1, 1, 1), padding='same', name='Conv3d_2b_1x1')
x = conv3d_bn(x, 192, 3, 3, 3, strides=(1, 1, 1), padding='same', name='Conv3d_2c_3x3')

# Downsampling (spatial only)
x = MaxPooling3D((1, 3, 3), strides=(1, 2, 2), padding='same', name='MaxPool2d_3a_3x3')(x)

# Mixed 3b
branch_0 = conv3d_bn(x, 64, 1, 1, 1, padding='same', name='Conv3d_3b_0a_1x1')

branch_1 = conv3d_bn(x, 96, 1, 1, 1, padding='same', name='Conv3d_3b_1a_1x1')
branch_1 = conv3d_bn(branch_1, 128, 3, 3, 3, padding='same', name='Conv3d_3b_1b_3x3')

branch_2 = conv3d_bn(x, 16, 1, 1, 1, padding='same', name='Conv3d_3b_2a_1x1')
branch_2 = conv3d_bn(branch_2, 32, 3, 3, 3, padding='same', name='Conv3d_3b_2b_3x3')

branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_3b_3a_3x3')(x)
branch_3 = conv3d_bn(branch_3, 32, 1, 1, 1, padding='same', name='Conv3d_3b_3b_1x1')

x = layers.concatenate(
    [branch_0, branch_1, branch_2, branch_3],
    axis=channel_axis,
    name='Mixed_3b')

# Mixed 3c
branch_0 = conv3d_bn(x, 128, 1, 1, 1, padding='same', name='Conv3d_3c_0a_1x1')

branch_1 = conv3d_bn(x, 128, 1, 1, 1, padding='same', name='Conv3d_3c_1a_1x1')
branch_1 = conv3d_bn(branch_1, 192, 3, 3, 3, padding='same', name='Conv3d_3c_1b_3x3')

branch_2 = conv3d_bn(x, 32, 1, 1, 1, padding='same', name='Conv3d_3c_2a_1x1')
branch_2 = conv3d_bn(branch_2, 96, 3, 3, 3, padding='same', name='Conv3d_3c_2b_3x3')

branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_3c_3a_3x3')(x)
branch_3 = conv3d_bn(branch_3, 64, 1, 1, 1, padding='same', name='Conv3d_3c_3b_1x1')

x = layers.concatenate(
    [branch_0, branch_1, branch_2, branch_3],
    axis=channel_axis,
    name='Mixed_3c')


# Downsampling (spatial and temporal)
x = MaxPooling3D((3, 3, 3), strides=(2, 2, 2), padding='same', name='MaxPool2d_4a_3x3')(x)

# Mixed 4b
branch_0 = conv3d_bn(x, 192, 1, 1, 1, padding='same', name='Conv3d_4b_0a_1x1')

branch_1 = conv3d_bn(x, 96, 1, 1, 1, padding='same', name='Conv3d_4b_1a_1x1')
branch_1 = conv3d_bn(branch_1, 208, 3, 3, 3, padding='same', name='Conv3d_4b_1b_3x3')

branch_2 = conv3d_bn(x, 16, 1, 1, 1, padding='same', name='Conv3d_4b_2a_1x1')
branch_2 = conv3d_bn(branch_2, 48, 3, 3, 3, padding='same', name='Conv3d_4b_2b_3x3')

branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_4b_3a_3x3')(x)
branch_3 = conv3d_bn(branch_3, 64, 1, 1, 1, padding='same', name='Conv3d_4b_3b_1x1')

x = layers.concatenate(
    [branch_0, branch_1, branch_2, branch_3],
    axis=channel_axis,
    name='Mixed_4b')

# Mixed 4c
branch_0 = conv3d_bn(x, 160, 1, 1, 1, padding='same', name='Conv3d_4c_0a_1x1')

branch_1 = conv3d_bn(x, 112, 1, 1, 1, padding='same', name='Conv3d_4c_1a_1x1')
branch_1 = conv3d_bn(branch_1, 224, 3, 3, 3, padding='same', name='Conv3d_4c_1b_3x3')

branch_2 = conv3d_bn(x, 24, 1, 1, 1, padding='same', name='Conv3d_4c_2a_1x1')
branch_2 = conv3d_bn(branch_2, 64, 3, 3, 3, padding='same', name='Conv3d_4c_2b_3x3')

branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_4c_3a_3x3')(x)
branch_3 = conv3d_bn(branch_3, 64, 1, 1, 1, padding='same', name='Conv3d_4c_3b_1x1')

x = layers.concatenate(
    [branch_0, branch_1, branch_2, branch_3],
    axis=channel_axis,
    name='Mixed_4c')

# Mixed 4d
branch_0 = conv3d_bn(x, 128, 1, 1, 1, padding='same', name='Conv3d_4d_0a_1x1')

branch_1 = conv3d_bn(x, 128, 1, 1, 1, padding='same', name='Conv3d_4d_1a_1x1')
branch_1 = conv3d_bn(branch_1, 256, 3, 3, 3, padding='same', name='Conv3d_4d_1b_3x3')

branch_2 = conv3d_bn(x, 24, 1, 1, 1, padding='same', name='Conv3d_4d_2a_1x1')
branch_2 = conv3d_bn(branch_2, 64, 3, 3, 3, padding='same', name='Conv3d_4d_2b_3x3')

branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_4d_3a_3x3')(x)
branch_3 = conv3d_bn(branch_3, 64, 1, 1, 1, padding='same', name='Conv3d_4d_3b_1x1')

x = layers.concatenate(
    [branch_0, branch_1, branch_2, branch_3],
    axis=channel_axis,
    name='Mixed_4d')

# Mixed 4e
branch_0 = conv3d_bn(x, 112, 1, 1, 1, padding='same', name='Conv3d_4e_0a_1x1')

branch_1 = conv3d_bn(x, 144, 1, 1, 1, padding='same', name='Conv3d_4e_1a_1x1')
branch_1 = conv3d_bn(branch_1, 288, 3, 3, 3, padding='same', name='Conv3d_4e_1b_3x3')

branch_2 = conv3d_bn(x, 32, 1, 1, 1, padding='same', name='Conv3d_4e_2a_1x1')
branch_2 = conv3d_bn(branch_2, 64, 3, 3, 3, padding='same', name='Conv3d_4e_2b_3x3')

branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_4e_3a_3x3')(x)
branch_3 = conv3d_bn(branch_3, 64, 1, 1, 1, padding='same', name='Conv3d_4e_3b_1x1')

x = layers.concatenate(
    [branch_0, branch_1, branch_2, branch_3],
    axis=channel_axis,
    name='Mixed_4e')

# Mixed 4f
branch_0 = conv3d_bn(x, 256, 1, 1, 1, padding='same', name='Conv3d_4f_0a_1x1')

branch_1 = conv3d_bn(x, 160, 1, 1, 1, padding='same', name='Conv3d_4f_1a_1x1')
branch_1 = conv3d_bn(branch_1, 320, 3, 3, 3, padding='same', name='Conv3d_4f_1b_3x3')

branch_2 = conv3d_bn(x, 32, 1, 1, 1, padding='same', name='Conv3d_4f_2a_1x1')
branch_2 = conv3d_bn(branch_2, 128, 3, 3, 3, padding='same', name='Conv3d_4f_2b_3x3')

branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_4f_3a_3x3')(x)
branch_3 = conv3d_bn(branch_3, 128, 1, 1, 1, padding='same', name='Conv3d_4f_3b_1x1')

x = layers.concatenate(
    [branch_0, branch_1, branch_2, branch_3],
    axis=channel_axis,
    name='Mixed_4f')


# Downsampling (spatial and temporal)
x = MaxPooling3D((2, 2, 2), strides=(2, 2, 2), padding='same', name='MaxPool2d_5a_2x2')(x)

# Mixed 5b
branch_0 = conv3d_bn(x, 256, 1, 1, 1, padding='same', name='Conv3d_5b_0a_1x1')

branch_1 = conv3d_bn(x, 160, 1, 1, 1, padding='same', name='Conv3d_5b_1a_1x1')
branch_1 = conv3d_bn(branch_1, 320, 3, 3, 3, padding='same', name='Conv3d_5b_1b_3x3')

branch_2 = conv3d_bn(x, 32, 1, 1, 1, padding='same', name='Conv3d_5b_2a_1x1')
branch_2 = conv3d_bn(branch_2, 128, 3, 3, 3, padding='same', name='Conv3d_5b_2b_3x3')

branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_5b_3a_3x3')(x)
branch_3 = conv3d_bn(branch_3, 128, 1, 1, 1, padding='same', name='Conv3d_5b_3b_1x1')

x = layers.concatenate(
    [branch_0, branch_1, branch_2, branch_3],
    axis=channel_axis,
    name='Mixed_5b')

# Mixed 5c
branch_0 = conv3d_bn(x, 384, 1, 1, 1, padding='same', name='Conv3d_5c_0a_1x1')

branch_1 = conv3d_bn(x, 192, 1, 1, 1, padding='same', name='Conv3d_5c_1a_1x1')
branch_1 = conv3d_bn(branch_1, 384, 3, 3, 3, padding='same', name='Conv3d_5c_1b_3x3')

branch_2 = conv3d_bn(x, 48, 1, 1, 1, padding='same', name='Conv3d_5c_2a_1x1')
branch_2 = conv3d_bn(branch_2, 128, 3, 3, 3, padding='same', name='Conv3d_5c_2b_3x3')

branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_5c_3a_3x3')(x)
branch_3 = conv3d_bn(branch_3, 128, 1, 1, 1, padding='same', name='Conv3d_5c_3b_1x1')

x = layers.concatenate(
    [branch_0, branch_1, branch_2, branch_3],
    axis=channel_axis,
    name='Mixed_5c')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
if include_top:
    # Classification block
    x = AveragePooling3D((2, 7, 7), strides=(1, 1, 1), padding='valid', name='global_avg_pool')(x)
    x = Dropout(dropout_prob)(x)

    x = conv3d_bn(x, classes, 1, 1, 1, padding='same', 
            use_bias=True, use_activation_fn=False, use_bn=False, name='Conv3d_6a_1x1')

    num_frames_remaining = int(x.shape[1])
    x = Reshape((num_frames_remaining, classes))(x)

    # logits (raw scores for each class)
    x = Lambda(lambda x: K.mean(x, axis=1, keepdims=False),
                output_shape=lambda s: (s[0], s[2]))(x)

    if not endpoint_logit:
        x = Activation('softmax', name='prediction')(x)
else:
    h = int(x.shape[2])
    w = int(x.shape[3])
    x = AveragePooling3D((2, h, w), strides=(1, 1, 1), padding='valid', name='global_avg_pool')(x)

In [0]:
inputs = img_input
# create model
model = Model(inputs, x, name='i3d_inception')

# load weights
if weights in WEIGHTS_NAME:
    if weights == WEIGHTS_NAME[0]:   # rgb_kinetics_only
        if include_top:
            weights_url = WEIGHTS_PATH['rgb_kinetics_only']
            model_name = 'i3d_inception_rgb_kinetics_only.h5'
        else:
            weights_url = WEIGHTS_PATH_NO_TOP['rgb_kinetics_only']
            model_name = 'i3d_inception_rgb_kinetics_only_no_top.h5'

    elif weights == WEIGHTS_NAME[1]: # flow_kinetics_only
        if include_top:
            weights_url = WEIGHTS_PATH['flow_kinetics_only']
            model_name = 'i3d_inception_flow_kinetics_only.h5'
        else:
            weights_url = WEIGHTS_PATH_NO_TOP['flow_kinetics_only']
            model_name = 'i3d_inception_flow_kinetics_only_no_top.h5'

    elif weights == WEIGHTS_NAME[2]: # rgb_imagenet_and_kinetics
        if include_top:
            weights_url = WEIGHTS_PATH['rgb_imagenet_and_kinetics']
            model_name = 'i3d_inception_rgb_imagenet_and_kinetics.h5'
        else:
            weights_url = WEIGHTS_PATH_NO_TOP['rgb_imagenet_and_kinetics']
            model_name = 'i3d_inception_rgb_imagenet_and_kinetics_no_top.h5'

    elif weights == WEIGHTS_NAME[3]: # flow_imagenet_and_kinetics
        if include_top:
            weights_url = WEIGHTS_PATH['flow_imagenet_and_kinetics']
            model_name = 'i3d_inception_flow_imagenet_and_kinetics.h5'
        else:
            weights_url = WEIGHTS_PATH_NO_TOP['flow_imagenet_and_kinetics']
            model_name = 'i3d_inception_flow_imagenet_and_kinetics_no_top.h5'

In [0]:
# #model.summary()

In [44]:
# load weights
if weights in WEIGHTS_NAME:
    if weights == WEIGHTS_NAME[0]:   # rgb_kinetics_only
        if include_top:
            weights_url = WEIGHTS_PATH['rgb_kinetics_only']
            model_name = 'i3d_inception_rgb_kinetics_only.h5'
        else:
            weights_url = WEIGHTS_PATH_NO_TOP['rgb_kinetics_only']
            model_name = 'i3d_inception_rgb_kinetics_only_no_top.h5'

    elif weights == WEIGHTS_NAME[1]: # flow_kinetics_only
        if include_top:
            weights_url = WEIGHTS_PATH['flow_kinetics_only']
            model_name = 'i3d_inception_flow_kinetics_only.h5'
        else:
            weights_url = WEIGHTS_PATH_NO_TOP['flow_kinetics_only']
            model_name = 'i3d_inception_flow_kinetics_only_no_top.h5'

    elif weights == WEIGHTS_NAME[2]: # rgb_imagenet_and_kinetics
        if include_top:
            weights_url = WEIGHTS_PATH['rgb_imagenet_and_kinetics']
            model_name = 'i3d_inception_rgb_imagenet_and_kinetics.h5'
        else:
            weights_url = WEIGHTS_PATH_NO_TOP['rgb_imagenet_and_kinetics']
            model_name = 'i3d_inception_rgb_imagenet_and_kinetics_no_top.h5'

    elif weights == WEIGHTS_NAME[3]: # flow_imagenet_and_kinetics
        if include_top:
            weights_url = WEIGHTS_PATH['flow_imagenet_and_kinetics']
            model_name = 'i3d_inception_flow_imagenet_and_kinetics.h5'
        else:
            weights_url = WEIGHTS_PATH_NO_TOP['flow_imagenet_and_kinetics']
            model_name = 'i3d_inception_flow_imagenet_and_kinetics_no_top.h5'

    downloaded_weights_path = get_file(model_name, weights_url, cache_subdir='models')
    #model.load_weights(downloaded_weights_path)

    if K.backend() == 'theano':
        layer_utils.convert_all_kernels_in_model(model)

    if K.image_data_format() == 'channels_first' and K.backend() == 'tensorflow':
        warnings.warn('You are using the TensorFlow backend, yet you '
                      'are using the Theano '
                      'image data format convention '
                      '(`image_data_format="channels_first"`). '
                      'For best performance, set '
                      '`image_data_format="channels_last"` in '
                      'your keras config '
                      'at ~/.keras/keras.json.')

elif weights is not None:
    model.load_weights(weights)

49512448/49507528 [==============================] - 2s 0us/step


In [0]:
# #model.summary()

In [47]:
print("Add top layers with %d output classes ..." % oClasses.nClasses)
keI3DOflow = layers_freeze(model)
#keI3DOflow.summary()

Add top layers with 3 output classes ...
Freeze all 195 layers in Model i3d_inception


In [49]:
keI3DOflow = add_i3d_top(keI3DOflow, oClasses.nClasses, dropout_prob=0.5)
keI3DOflow.summary()

Model: "i3d_with_top"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 120, 224, 224 0                                            
__________________________________________________________________________________________________
Conv3d_1a_7x7_conv (Conv3D)     (None, 60, 112, 112, 65856       input_1[0][0]                    
__________________________________________________________________________________________________
Conv3d_1a_7x7_bn (BatchNormaliz (None, 60, 112, 112, 192         Conv3d_1a_7x7_conv[0][0]         
__________________________________________________________________________________________________
Conv3d_1a_7x7 (Activation)      (None, 60, 112, 112, 0           Conv3d_1a_7x7_bn[0][0]           
_______________________________________________________________________________________

/content/drive/My Drive/_main_/GoForward_/i3D_model/1c3d_2016/model_i3d.py:625: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., name="i3d_with_top", outputs=Tensor("i3...)`
  new_model = Model(inputs = base_model.input, output = predictions, name = "i3d_with_top")


In [50]:
# Prep logging
sLog = time.strftime("%Y%m%d-%H%M", time.gmtime()) + \
    "-%s%03d-oflow-i3d"%(diVideoSet["sName"], diVideoSet["nClasses"])
sLog

'20200419-1346-chalearn_img_flow_tvl1-fast003-oflow-i3d'

In [0]:
# Helper: Save results
csv_logger = keras.callbacks.CSVLogger("log/" + sLog + "-acc.csv", append = True)

# Helper: Save the model
os.makedirs(sModelDir, exist_ok=True)
cpTopLast = keras.callbacks.ModelCheckpoint(filepath = sModelDir + "/" + sLog + "-above-last.h5", 
                                            verbose = 0)

cpTopBest = keras.callbacks.ModelCheckpoint(filepath = sModelDir + "/" + sLog + "-above-best.h5",
                                            verbose = 1, save_best_only = True)

cpAllLast = keras.callbacks.ModelCheckpoint(filepath = sModelDir + "/" + sLog + "-entire-last.h5", 
                                            verbose = 0)

cpAllBest = keras.callbacks.ModelCheckpoint(filepath = sModelDir + "/" + sLog + "-entire-best.h5",
                                            verbose = 1, save_best_only = True)

In [52]:
cpAllBest

In [53]:
# Fit top layers
print("Fit I3D top layers with generator: %s" % (diTrainTop))
optimizer = keras.optimizers.Adam(lr = diTrainTop["fLearn"])
keI3DOflow.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
count_params(keI3DOflow) 

Fit I3D top layers with generator: {'fLearn': 0.001, 'nEpochs': 3}
Total params: 12,297,619
Trainable params: 3,075
Non-trainable params: 12,294,544


In [54]:
keI3DOflowc(generator = genFramesTrain,
                          validation_data = genFramesVal,
                          epochs = diTrainTop["nEpochs"],
                          workers = 4,
                          use_multiprocessing = True,
                          max_queue_size = 8,
                          verbose = 1,
                          callbacks=[csv_logger, 
                                    cpTopLast, 
                                    cpTopBest])


Epoch 1/3


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 6 could not be retrieved. It could be because a worker has died.
  UserWarning)


 2/15 [===>..........................] - ETA: 21:24 - loss: 0.9491 - accuracy: 0.6250

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 3 could not be retrieved. It could be because a worker has died.
  UserWarning)


 4/15 [=======>......................] - ETA: 17:11 - loss: 1.1478 - accuracy: 0.5000

...

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 2 could not be retrieved. It could be because a worker has died.
  UserWarning)


14/15 [===========================>..] - ETA: 18s - loss: 1.2130 - accuracy: 0.3818

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 13 could not be retrieved. It could be because a worker has died.
  UserWarning)


15/15 [==============================] - 407s 27s/step - loss: 1.1988 - accuracy: 0.3729 - val_loss: 1.0979 - val_accuracy: 0.3571


OSError: ignored

In [0]:
# Fit entire I3D model
print("Finetune all I3D layers with generator: %s" % (diTrainAll))
keI3DOflow = layers_unfreeze(keI3DOflow)
optimizer = keras.optimizers.Adam(lr = diTrainAll["fLearn"])
keI3DOflow.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
count_params(keI3DOflow)    

keI3DOflow.fit_generator(generator = genFramesTrain, 
                          validation_data = genFramesVal,
                          epochs = diTrainAll["nEpochs"],
                          workers = 4,                 
                          use_multiprocessing = True,
                          max_queue_size = 8, 
                          verbose = 1,
                          callbacks=[csv_logger, 
                                    cpAllLast, 
                                    cpAllBest])